<a href="https://colab.research.google.com/github/dokdin/Portfolio3/blob/master/LG_AI_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 파일 디렉토리 
PATH='/content/drive/MyDrive/Data/LG/'
CSV_PATH='/content/drive/MyDrive/Data/LG/CSV/'

In [3]:
!pip install pycaret

     |████████████████████████████████| 256kB 7.2MB/s 
     |████████████████████████████████| 245kB 11.1MB/s 
     |████████████████████████████████| 157.5MB 47kB/s 
     |████████████████████████████████| 1.8MB 56.3MB/s 
     |████████████████████████████████| 1.7MB 58.1MB/s 
     |████████████████████████████████| 276kB 56.1MB/s 
     |████████████████████████████████| 65.8MB 141kB/s 
     |████████████████████████████████| 174kB 59.8MB/s 
     |████████████████████████████████| 14.2MB 254kB/s 
     |████████████████████████████████| 6.8MB 55.1MB/s 
     |████████████████████████████████| 102kB 14.5MB/s 
     |████████████████████████████████| 604kB 49.0MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
     |████████████████████████████████| 3.1MB 48.4MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 163kB 60.8MB/s 
     |████████████████████████████████| 81kB 12.1MB/s

In [4]:
# 필요 라이브러리 호출 

from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요 함수 정의

def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    # hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year,month, day)

In [5]:
# 파일 불러오기 
train_err = pd.read_csv(PATH+"train_err_data.csv")
train_quality = pd.read_csv(PATH+'train_quality_data.csv')
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
test_err = pd.read_csv(PATH+'test_err_data.csv')

# train data

In [6]:
# 시간 함수 적용하기 
# train_err['datetime']=train_err['time'].apply(make_datetime)
train_err['datetime']=train_err['time'].apply(make_datetime)

In [7]:
print(train_err['datetime'].min())
print(train_err['datetime'].max())

2020-10-31 00:00:00
2020-12-02 00:00:00


In [8]:
# 시간변경
filtered_df=train_err.loc[train_err["datetime"].between('2020-10-31','2020-11-30')]

In [9]:
# 시간데이터 수정 후 
print(str(train_err.shape))
print(str(filtered_df.shape))

(16554663, 7)
(16554582, 7)


In [10]:
# datetime에서 정수로 변환
filtered_df['datetime'] = filtered_df['datetime'].dt.strftime('%Y-%m-%d')

In [11]:
def test1(x):
    if x=='2020-11-01': return 1
    elif x=='2020-11-02': return 2
    elif x=='2020-11-03': return 3
    elif x=='2020-11-04': return 4
    elif x=='2020-11-06': return 5
    elif x=='2020-11-07': return 6
    elif x=='2020-11-08': return 7
    elif x=='2020-11-09': return 8
    elif x=='2020-11-10': return 9
    elif x=='2020-11-11': return 10
    elif x=='2020-11-12': return 11
    elif x=='2020-11-13': return 12
    elif x=='2020-11-14': return 13
    elif x=='2020-11-15': return 14
    elif x=='2020-11-16': return 15
    elif x=='2020-11-17': return 16
    elif x=='2020-11-18': return 17
    elif x=='2020-11-19': return 18
    elif x=='2020-11-20': return 19
    elif x=='2020-11-21': return 20
    elif x=='2020-11-22': return 21
    elif x=='2020-11-23': return 22
    elif x=='2020-11-24': return 23
    elif x=='2020-11-25': return 24
    elif x=='2020-11-26': return 25
    elif x=='2020-11-27': return 26
    elif x=='2020-11-28': return 27
    elif x=='2020-11-29': return 28
    elif x=='2020-11-30': return 29
    else: return 0

In [12]:
filtered_df['datetime']=filtered_df['datetime'].apply(test1)

In [13]:
# 일자 별 독립변수 생성 
id_error = filtered_df[['user_id','datetime']].values
errorr = np.zeros((15000,30))
for person_idx, err in tqdm(id_error):
    # person_idx - 10000 위치에 person_idx, errtype에 해당하는 error값을 +1
    errorr[person_idx - 10000,err - 1] += 1

100%|██████████| 16554582/16554582 [00:40<00:00, 410146.16it/s]


In [14]:
# errtype별 독립변수 생성 
id_errorr = filtered_df[['user_id','errtype']].values
error = np.zeros((15000,42))

for person_idx, err in tqdm(id_errorr):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - 10000,err - 1] += 1
error.shape

100%|██████████| 16554582/16554582 [00:39<00:00, 420320.33it/s]


(15000, 42)

In [15]:
# 데이터 프레임 화 
train = pd.DataFrame(data=error)
train2=pd.DataFrame(data=errorr)

In [16]:
train=train.merge(train2,left_index=True, right_index=True )

In [17]:
# 회원별 모델
train_err_model=filtered_df[['user_id','model_nm']].drop_duplicates()
model_list=np.sort(train_err_model['model_nm'].unique())
Model = pd.DataFrame()

for i in tqdm(range(len(model_list))):
  model=np.zeros((15000))
  train_err_model1=train_err_model[train_err_model['model_nm']==model_list[i]].reset_index(drop=True)
  model[train_err_model1.user_id.unique()-10000]=1
  Model[model_list[i]]=model

100%|██████████| 9/9 [00:00<00:00, 422.20it/s]


In [18]:
# 확인
train=train.merge(Model,left_index=True, right_index=True )

In [19]:
# 변수추가를 위한 종속변수 제거 
# train.drop(['problem'],axis='columns')

In [20]:
Number_model = pd.DataFrame(columns=['user_id', 'Number_of_model'])
user_list=filtered_df.user_id.unique().tolist()
for i in tqdm(range(len(user_list))):
  count=len(filtered_df.loc[filtered_df.user_id==user_list[i]].model_nm.unique())
  Number_model=Number_model.append(pd.DataFrame([[user_list[i],count]],columns=['user_id','Number_of_model']),ignore_index=True)

100%|██████████| 15000/15000 [04:46<00:00, 52.40it/s]


In [21]:
Number_model=Number_model.drop(['user_id'],axis='columns')

In [22]:
# 확인
train=train.merge(Number_model,left_index=True, right_index=True )

In [23]:
# 종속 변수
problem = np.zeros(15000)
problem[train_prob.user_id.unique()-10000] = 1 

train['problem'] = problem
# del error, problem

In [24]:
train

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,24_x,25_x,26_x,27_x,28_x,29_x,30,31,32,33,34,35,36,37,38,39,40,41,0_y,1_y,2_y,3_y,4_y,5_y,6_y,7_y,8_y,9_y,10_y,11_y,12_y,13_y,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y,24_y,25_y,26_y,27_y,28_y,29_y,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,Number_of_model,problem
0,0.0,0.0,8.0,104.0,0.0,1.0,1.0,0.0,0.0,7.0,15.0,16.0,1.0,10.0,59.0,61.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,9.0,18.0,5.0,9.0,20.0,7.0,5.0,10.0,14.0,5.0,17.0,9.0,16.0,9.0,8.0,17.0,19.0,5.0,7.0,5.0,5.0,5.0,9.0,11.0,12.0,18.0,13.0,9.0,10.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0.0
1,0.0,0.0,0.0,0.0,53.0,1.0,1.0,0.0,0.0,0.0,10.0,11.0,1.0,15.0,151.0,128.0,0.0,4.0,1.0,3.0,1.0,756.0,751.0,5.0,1.0,22.0,0.0,0.0,0.0,0.0,250.0,0.0,10.0,18.0,0.0,1.0,1.0,0.0,0.0,113.0,56.0,1.0,11.0,50.0,29.0,48.0,29.0,13.0,24.0,53.0,52.0,23.0,39.0,8.0,41.0,24.0,15.0,30.0,25.0,27.0,45.0,25.0,18.0,18.0,27.0,11.0,58.0,1452.0,38.0,62.0,28.0,42.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0
2,0.0,0.0,2.0,132.0,1.0,2.0,1.0,0.0,0.0,1.0,13.0,14.0,1.0,4.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,25.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,13.0,13.0,15.0,8.0,17.0,11.0,12.0,10.0,11.0,11.0,5.0,15.0,13.0,15.0,5.0,9.0,15.0,5.0,10.0,16.0,3.0,6.0,11.0,5.0,16.0,0.0,8.0,9.0,9.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0.0
3,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,52.0,30.0,1.0,0.0,0.0,0.0,0.0,28.0,19.0,0.0,0.0,59.0,0.0,0.0,0.0,0.0,65.0,0.0,8.0,0.0,0.0,1.0,1.0,2.0,0.0,17.0,1.0,0.0,9.0,14.0,10.0,5.0,5.0,2.0,11.0,5.0,19.0,24.0,6.0,6.0,2.0,2.0,15.0,6.0,7.0,15.0,5.0,7.0,21.0,11.0,17.0,6.0,8.0,21.0,11.0,14.0,6.0,16.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
4,0.0,0.0,0.0,1.0,0.0,3.0,4.0,0.0,0.0,0.0,16.0,19.0,3.0,5.0,143.0,91.0,0.0,0.0,0.0,0.0,0.0,140.0,119.0,0.0,0.0,33.0,0.0,0.0,0.0,0.0,176.0,0.0,16.0,0.0,0.0,1.0,1.0,0.0,0.0,4.0,0.0,2.0,25.0,21.0,49.0,28.0,28.0,18.0,29.0,20.0,54.0,19.0,21.0,17.0,20.0,15.0,15.0,18.0,41.0,8.0,102.0,28.0,16.0,18.0,23.0,27.0,19.0,24.0,18.0,23.0,22.0,11.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0.0,0.0,0.0,0.0,2.0,5.0,5.0,0.0,0.0,0.0,5.0,10.0,8.0,8.0,26.0,18.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0,0.0,0.0,22.0,0.0,0.0,0.0,0.0,31.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,7.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,17.0,17.0,18.0,0.0,24.0,3.0,30.0,22.0,17.0,22.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0
14996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0.0
14997,0.0,0.0,0.0,1.0,8.0,1.0,1.0,0.0,0.0,0.0,16.0,18.0,4.0,1.0,181.0,138.0,0.0,0.0,0.0,0.0,0.0,50.0,29.0,0.0,0.0,49.0,0.0,0.0,0.0,0.0,223.0,0.0,16.0,17.0,0.0,1.0,1.0,0.0,0.0,58.0,8.0,5.0,9.0,34.0,34.0,64.0,31.0,24.0,30.0,26.0,18.0,9.0,19.0,19.0,38.0,31.0,10.0,27.0,19.0,32.0,29.0,25.0,29.0,42.0,33.0,41.0,23.0,30.0,31.0,34.0,13.0,22.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0
14998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,15.0,0.0,1.0,51.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,14.0,0.0,15.0,12.0,0.0,1.0,1.0,0.0,0.0,6.0,0.0,0.0,7.0,0.0,10.0,0.0,0.0,8.0,0.0,7.0,37.0,7.0,0.0,6.0,2.0,5.0,6.0,0.0,9.0,1.0,6.0,2.0,5.0,6.0,0.0,6.0,0.0,7.0,0.0,7.0,4.0,7.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0


In [25]:
# AUTOML 실행
clf = setup(data = train, target = 'problem')

,Description,Value
0,session_id,7436
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 83)"
5,Missing Values,False
6,Numeric Features,72
7,Categorical Features,10
8,Ordinal Features,False
9,High Cardinality Features,False


In [26]:
best_5  = compare_models(sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7969,0.8166,0.5053,0.8130,0.6228,0.4942,0.5205,14.879
gbc,Gradient Boosting Classifier,0.7946,0.8138,0.4818,0.8279,0.6087,0.4825,0.5151,4.280
et,Extra Trees Classifier,0.7893,0.8129,0.4875,0.8001,0.6056,0.4734,0.5005,1.550
rf,Random Forest Classifier,0.7928,0.8107,0.4804,0.8221,0.6061,0.4785,0.5104,2.551
lightgbm,Light Gradient Boosting Machine,0.7923,0.8100,0.5128,0.7883,0.6209,0.4867,0.5082,0.574
ada,Ada Boost Classifier,0.7823,0.7975,0.4827,0.7775,0.5949,0.4573,0.4818,0.909
xgboost,Extreme Gradient Boosting,0.7838,0.7892,0.5380,0.7406,0.6228,0.4767,0.4889,10.693
lr,Logistic Regression,0.7493,0.7554,0.3540,0.7658,0.4835,0.3466,0.3918,1.576
lda,Linear Discriminant Analysis,0.7639,0.7543,0.3617,0.8339,0.5037,0.3795,0.4367,0.159
qda,Quadratic Discriminant Analysis,0.7384,0.7387,0.3634,0.7058,0.4790,0.3280,0.3604,0.092


In [27]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7952,0.8169,0.4885,0.8237,0.6133,0.4861,0.5168
1,0.7881,0.8173,0.4799,0.8034,0.6009,0.4688,0.4977
2,0.7881,0.8140,0.4656,0.8186,0.5936,0.4646,0.4984
3,0.8110,0.8373,0.5372,0.8352,0.6539,0.5321,0.5566
4,0.7932,0.8273,0.4763,0.8279,0.6047,0.4782,0.5117
Mean,0.7951,0.8226,0.4895,0.8218,0.6133,0.4860,0.5162
SD,0.0084,0.0086,0.0250,0.0107,0.0213,0.0243,0.0215


In [28]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.784,0.8157,0.4719,0.8038,0.5947,0.4607,0.4911


In [29]:
final_model = finalize_model(blended)

In [30]:
final_model

TunableVotingClassifier(estimators=[('catboost',
                                     <catboost.core.CatBoostClassifier object at 0x7f4a17ad1780>),
                                    ('gbc',
                                     GradientBoostingClassifier(ccp_alpha=0.0,
                                                                criterion='friedman_mse',
                                                                init=None,
                                                                learning_rate=0.1,
                                                                loss='deviance',
                                                                max_depth=3,
                                                                max_features=None,
                                                                max_leaf_nodes=None,
                                                                min_impurity_decrease=0.0,
                                                                

# test data

In [31]:
test_err = pd.read_csv(PATH+'test_err_data.csv')

In [32]:
# 시간 함수 적용하기 
test_err['datetime']=test_err['time'].apply(make_datetime)

In [33]:
# datetime에서 str로 변환
test_err['datetime'] = test_err['datetime'].dt.strftime('%Y-%m-%d')

In [34]:
test_err=test_err.loc[test_err["datetime"].between('2020-10-31','2020-11-30')]
test_err=test_err.drop(['time'],axis='columns')

In [35]:
test_err.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16532576 entries, 0 to 16532647
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   model_nm  object
 2   fwver     object
 3   errtype   int64 
 4   errcode   object
 5   datetime  object
dtypes: int64(2), object(4)
memory usage: 882.9+ MB


In [36]:
def test1(x):
    if x=='2020-11-01': return 1
    elif x=='2020-11-02': return 2
    elif x=='2020-11-03': return 3
    elif x=='2020-11-04': return 4
    elif x=='2020-11-06': return 5
    elif x=='2020-11-07': return 6
    elif x=='2020-11-08': return 7
    elif x=='2020-11-09': return 8
    elif x=='2020-11-10': return 9
    elif x=='2020-11-11': return 10
    elif x=='2020-11-12': return 11
    elif x=='2020-11-13': return 12
    elif x=='2020-11-14': return 13
    elif x=='2020-11-15': return 14
    elif x=='2020-11-16': return 15
    elif x=='2020-11-17': return 16
    elif x=='2020-11-18': return 17
    elif x=='2020-11-19': return 18
    elif x=='2020-11-20': return 19
    elif x=='2020-11-21': return 20
    elif x=='2020-11-22': return 21
    elif x=='2020-11-23': return 22
    elif x=='2020-11-24': return 23
    elif x=='2020-11-25': return 24
    elif x=='2020-11-26': return 25
    elif x=='2020-11-27': return 26
    elif x=='2020-11-28': return 27
    elif x=='2020-11-29': return 28
    elif x=='2020-11-30': return 29
    else: return 0

In [37]:
test_err['datetime']=test_err['datetime'].apply(test1)

In [38]:
test_err.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16532576 entries, 0 to 16532647
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   model_nm  object
 2   fwver     object
 3   errtype   int64 
 4   errcode   object
 5   datetime  int64 
dtypes: int64(3), object(3)
memory usage: 882.9+ MB


In [39]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((14999,42))
for person_idx, err in tqdm(id_error):
    test_x[person_idx - 30000,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
test = pd.DataFrame(data=test_x)

100%|██████████| 16532576/16532576 [00:38<00:00, 426599.46it/s]


In [40]:
id_error = test_err[['user_id','datetime']].values
test_x = np.zeros((14999,30))
for person_idx, err in tqdm(id_error):
    # person_idx - 10000 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x[person_idx - 30000,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
testt = pd.DataFrame(data=test_x)

100%|██████████| 16532576/16532576 [00:38<00:00, 426432.31it/s]


In [41]:
 test2=test.merge(testt,left_index=True, right_index=True )

In [42]:
test_err_model=test_err[['user_id','model_nm']].drop_duplicates()
model_list=np.sort(test_err_model['model_nm'].unique())
Model = pd.DataFrame()

for i in tqdm(range(len(model_list))):
  model=np.zeros((14999))
  train_err_model1=test_err_model[test_err_model['model_nm']==model_list[i]].reset_index(drop=True)
  model[train_err_model1.user_id.unique()-30000]=1
  Model[model_list[i]]=model


100%|██████████| 9/9 [00:00<00:00, 340.84it/s]


In [43]:
 test3=test2.merge(Model,left_index=True, right_index=True )

In [44]:
Number_model = pd.DataFrame(columns=['user_id', 'Number_of_model'])
user_list=test_err.user_id.unique().tolist()
for i in tqdm(range(len(user_list))):
  count=len(test_err.loc[test_err.user_id==user_list[i]].model_nm.unique())
  Number_model=Number_model.append(pd.DataFrame([[user_list[i],count]],columns=['user_id','Number_of_model']),ignore_index=True)

100%|██████████| 14998/14998 [04:46<00:00, 52.43it/s]


In [45]:
new_data={'user_id':'43262','Number_of_model':0}
Number_model=Number_model.append(new_data,ignore_index=True)

In [46]:
 test4=test3.merge(Number_model,left_index=True, right_index=True )

In [47]:
predictions = predict_model(final_model, data = test4)

In [48]:
#pycaret에서는 score이 label을 맞출 확률이기때문에 output을 제출 양식에 맞게 바꿔줍니다
x = []
for i in range(len(predictions['Score'])):
  if predictions['Label'][i] =='1.0':
    x.append(predictions['Score'][i])
  else:
    x.append(1-predictions['Score'][i])


In [49]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = x
sample_submssion.to_csv(CSV_PATH+"210203(3).csv", index = False)